In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [22]:
# 생성된 샘플
examples = [
    {
        "movie": "Eternal Symphony",
        "answer": """
        Here is what I know:
        Director: Claire Howard
        Cast: Emma Stone, Tom Hardy
        Budget: $70,000,000
        Box Office: $210,000,000
        Genre: Drama, Fantasy
        Synopsis: A pianist with amnesia searches for a melody tied to their past in a mystical city.
        """
    },
    {
        "movie": "Starlight Horizon",
        "answer": """
        Here is what I know:
        Director: Ian Lee
        Cast: John Boyega, Lupita Nyong'o
        Budget: $90,000,000
        Box Office: $350,000,000
        Genre: Sci-Fi, Action
        Synopsis: A global team fights to stop an alien invasion in this sci-fi blockbuster.
        """
    },
    {
        "movie": "Crimson Echoes",
        "answer": """
        Here is what I know:
        Director: Sophia Delmar
        Cast: Keanu Reeves, Zendaya
        Budget: $60,000,000
        Box Office: $180,000,000
        Genre: Thriller, Mystery
        Synopsis: A retired detective and a mysterious girl unravel a series of murders in a city.
        """
    },
]

In [ ]:

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about the movie {movie}?"),
    ("ai", "{answer}")
])
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
# you give short answers. <- 할루시네이션 방지.
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about the movie {movie}?")
])

chain = final_prompt | chat

chain.invoke(
    {
        "movie": "Iron man"
    }
)


        Here is what I know:
        Director: Jon Favreau
        Cast: Robert Downey Jr., Gwyneth Paltrow
        Budget: $140,000,000
        Box Office: $585,000,000
        Genre: Action, Sci-Fi
        Synopsis: Billionaire inventor Tony Stark builds a powered suit of armor to fight evil forces.
        

AIMessageChunk(content='\n        Here is what I know:\n        Director: Jon Favreau\n        Cast: Robert Downey Jr., Gwyneth Paltrow\n        Budget: $140,000,000\n        Box Office: $585,000,000\n        Genre: Action, Sci-Fi\n        Synopsis: Billionaire inventor Tony Stark builds a powered suit of armor to fight evil forces.\n        ')